In [5]:
# 모델 환경 설정
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="nsmc",
    downstream_model_dir="nlpbook/generation",
    downstream_corpus_root_dir="content/Korpora",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [6]:
# 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

# 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='content/Korpora', downstream_model_dir='nlpbook/generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=16, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='content/Korpora', downstream_model_dir='nlpbook/generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=16, fp16=False, tpu_cores=0)
INFO:ratsnlp:Train

set seed: 7


In [7]:
# 말뭉치 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[Korpora] Corpus `nsmc` is already installed at c:\Users\Kang MinJae\Desktop\Code\algorithm_study\python\NLP\content\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at c:\Users\Kang MinJae\Desktop\Code\algorithm_study\python\NLP\content\Korpora\nsmc\ratings_test.txt


In [8]:
# 토크나이저 준비하기
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>"
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
# 학습 데이터셋 구축
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at content/Korpora\nsmc
INFO:ratsnlp:Creating features from dataset file at content/Korpora\nsmc
INFO:ratsnlp:Creating features from dataset file at content/Korpora\nsmc
INFO:ratsnlp:loading train data... LOOKING AT content/Korpora\nsmc\ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT content/Korpora\nsmc\ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT content/Korpora\nsmc\ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 3.978 s]
INFO:ratsnlp:tokenize sentences [took 3.978 s]
INFO:ratsnlp:tokenize sentences [took 3.978 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:s

In [11]:
train_dataset[0]

# input_ids: 극성 정보가 포함된 입력 문장을 토큰화한 뒤 인덱싱한 결과
# 뒤 쪽에 1은 eos_token(end of sentence)에 대응하며 패딩 역할
# attention_mask: 해당 토큰이 패딩 토큰인지 아닌지를 나타냄
# token_type_ids: 전체가 동일한 문서이므로 세그먼트 정보는 모두 0
# labels: input_ids와 같음

GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
# 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [14]:
# 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

INFO:ratsnlp:Creating features from dataset file at content/Korpora\nsmc
INFO:ratsnlp:Creating features from dataset file at content/Korpora\nsmc
INFO:ratsnlp:Creating features from dataset file at content/Korpora\nsmc
INFO:ratsnlp:loading test data... LOOKING AT content/Korpora\nsmc\ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT content/Korpora\nsmc\ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT content/Korpora\nsmc\ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 1.798 s]
INFO:ratsnlp:tokenize sentences [took 1.798 s]
INFO:ratsnlp:tokenize sentences [took 1.798 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁

In [15]:
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [16]:
# 모델 초기화
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name,
)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [17]:
# 태스크 정의
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [18]:
# 트레이너 정의
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
# 학습 개시
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

Missing logger folder: c:\Users\Kang MinJae\Desktop\Code\algorithm_study\python\NLP\nlpbook\generation\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Kang MinJae\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\core\optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

c:\Users\Kang MinJae\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000220A2B36B90>
Traceback (most recent call last):
  File "c:\Users\Kang MinJae\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "c:\Users\Kang MinJae\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1468, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
